In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 🔹 GPU 메모리 그로스
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print(f"GPU enabled with memory growth: {gpus}")
    except Exception as e:
        print("Failed to set memory growth:", e)

# 1) 데이터
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = (x_train / 255.0).astype('float32')[..., None]   # (N,28,28,1)
x_test  = (x_test  / 255.0).astype('float32')[..., None]

# 2) 모델
model = models.Sequential([
    layers.Conv2D(32, 3, padding='same', use_bias=False, input_shape=(28,28,1)),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPooling2D(), layers.Dropout(0.15),

    layers.Conv2D(64, 3, padding='same', use_bias=False),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPooling2D(), layers.Dropout(0.20),

    layers.Conv2D(128, 3, padding='same', use_bias=False),
    layers.BatchNormalization(), layers.Activation('relu'),

    layers.GlobalAveragePooling2D(),          # Flatten 대체
    layers.Dropout(0.30),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3) 콜백
cb = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)
]

# 4) 학습
history = model.fit(
    x_train, y_train,
    epochs=30,              # EarlyStopping이 최적 epoch에서 멈춤
    batch_size=64,          # 128→64: 일반화에 유리한 경우多
    validation_split=0.1,
    callbacks=cb,
    verbose=2
)

# 5) 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy (improved): {test_acc:.4f}")

# 6) 예측 예시
pred = model.predict(x_test[:5])
print("예측 결과:", np.argmax(pred, axis=1))
print("실제 레이블:", y_test[:5])


Epoch 1/30


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models

# 🔹 GPU 메모리 그로스 설정 (있을 때만)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print(f"GPU enabled with memory growth: {gpus}")
    except Exception as e:
        print("Failed to set memory growth:", e)

# 1️⃣ 데이터 로드
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 2️⃣ 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

# 3️⃣ 채널 차원 추가
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shaphape)  # (60000, 28, 28, 1)
print("x_test shape:", x_test.shape)    # (10000, 28, 28, 1)

# 4️⃣ CNN 모델 구성 (원 코드 유지)
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# 5️⃣ 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 6️⃣ 학습
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

# 7️⃣ 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

# 8️⃣ 예측 예시
predictions = model.predict(x_test[:5])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:5])


